# Information Entropy

## 1. Understanding Information Entropy
**Information entropy**, introduced by Claude Shannon, _is a measure of the uncertainty or randomness in a set of possible outcomes_. In the context of Wordle, entropy quantifies the expected information you would gain from making a particular guess, based on how it partitions the remaining possible words.

### Why Use Entropy in Wordle?

In Wordle, each guess provides feedback that narrows down the list of possible answers. By choosing a word that maximizes the expected information gain (entropy), you can eliminate the largest number of potential words, leading you closer to the solution more efficiently.

**Key Concepts:**
- **Probability Distribution:** The likelihood of each possible outcome.
- **Expected Information Gain:** The average amount of information you expect to gain from a guess.

## 2. Applying Entropy to Wordle

**How to Calculate Entropy for a Guess**
1. Possible Outcomes: For each guess, consider all possible feedback patterns (e.g., positions of green, yellow, and gray letters).

2. Partitioning the Word List: Each feedback pattern partitions the remaining possible words into subsets. Words that would produce the same feedback form a group.

3. Calculating Probabilities: For each feedback pattern, calculate the probability that it will occur, based on the current list of possible answers.

4. Entropy Formula:
$$\large
\text { Entropy }=-\sum_i p_i \log _2 p_i
$$
where $p_i$ is the probability of the $i$-th feedback pattern.


In [27]:
import math
import numpy as np
from collections import defaultdict, Counter


class Wordle:
    def __init__(self):
        self.ANSWERS_PATH = "../wordle/wordle-answers.txt"
        self.POSSIBLE_WORDS_PATH = "../wordle/wordle-possible-words.txt"
        # self.words = ["about", "blink", "crane", "drove", "merge", "rayon"]

    def load_words(self, file_path: str) -> list:
        with open(file_path, "r") as file:
            content = file.read()
            words = content.split("\n")
        if len(words[-1]) == 0:  # remove empty word after last word from
            words.pop()
        return words

    def get_letter_positions(self, word_played: str, color_letters) -> np.array:
        return np.array(
            [
                index
                for index, letter in enumerate(word_played)
                if letter in color_letters
            ]
        )

    # Function to find words in list of possible words in Wordle
    def find_words(
        self,
        word_played: str,
        green_letters: str,
        yellow_letters: str,
        gray_letters: str,
        answer_word_list: bool = True,
    ) -> list:
        """
        Function that takes the green and yellow letters and their positions and returns a list of possible words
        Args:
            :param word_played: string of word played
            :param green_letters: string of green letters
            :param yellow_letters: string of yellow letters
            :param gray_letters: string of bad letters
            :param answer_word_list: boolean to check if the possible words are the answer words
        Returns:
            :return: list of possible words
        """
        # List of possible words
        possible_words = []
        final_words = []

        # Get positions of green and yellow letters
        green_letter_positions = self.get_letter_positions(word_played, green_letters)
        yellow_letter_positions = self.get_letter_positions(word_played, yellow_letters)

        # Make all letters lower case
        green_letters = green_letters.lower()
        yellow_letters = yellow_letters.lower()
        gray_letters = gray_letters.lower()

        # Get list of words
        if answer_word_list:
            words = self.load_words(file_path=self.ANSWERS_PATH)
        else:
            words = self.load_words(file_path=self.POSSIBLE_WORDS_PATH)

        # Convert letters to sets for efficient checking
        set_yellow_letters = set(yellow_letters)
        set_bad_letters = set(gray_letters)

        possible_words = []
        # Iterate over each word
        for word in words:
            # Exclude words with bad letters
            if set_bad_letters & set(word):
                continue

            # Include words with yellow letters, but not at the specified positions
            if set_yellow_letters <= set(word) and all(
                word[pos] != yellow_letters[i]
                for i, pos in enumerate(yellow_letter_positions)
            ):
                possible_words.append(word)

        final_words = []
        # Filter words based on green letter positions
        if green_letters:
            for word in possible_words:
                if all(
                    word[pos] == green_letters[i]
                    for i, pos in enumerate(green_letter_positions)
                ):
                    final_words.append(word)
        else:
            final_words = possible_words
        self.words = final_words
        self.LEN_WORDS = len(self.words)
        return final_words

    def simulate_feedback_pattern(self, word_played: str) -> dict:
        # hash map of the feedback pattern
        feedback_patern = defaultdict(list)
        for word in self.words:
            for letter in word_played:
                if letter not in word:
                    feedback_patern[word].append("gray")
                elif letter in word and word_played.index(letter) != word.index(letter):
                    feedback_patern[word].append("yellow")
                elif letter in word_played and word_played.index(letter) == word.index(
                    letter
                ):
                    feedback_patern[word].append("green")
        return feedback_patern

    # calculate probabilities of feedback pattern
    def calculate_probabilities(self, feedback_patern) -> dict:
        # count the number of each feedback pattern
        list_counts = Counter(tuple(lst) for lst in feedback_patern.values())
        # calculate the probabilities of each feedback pattern
        probabilities = {}
        for key, value in list_counts.items():
            probabilities[key] = value / self.LEN_WORDS
        return probabilities

    # calculate the entropy of the probabilities
    def compute_entropy(self, probabilities: dict) -> float:
        entropy = 0
        for prob in probabilities.values():
            entropy += -prob * math.log2(prob)
        return entropy

    # compute entropy for the guess
    def compute_entropy_guess(self, word_played: str):
        feedback_patern = self.simulate_feedback_pattern(word_played)
        probabilities = self.calculate_probabilities(feedback_patern)
        entropy = self.compute_entropy(probabilities)
        self.entropy = entropy

In [28]:
# Initialize Wordle class
wordle = Wordle()

# Main function that returns a list of possible words
word_played = "crane"
green_letters = ""
yellow_letters = "e"
bad_letters = ""

# Find words in a list of possible words
words = wordle.find_words(word_played, green_letters, yellow_letters, bad_letters)
print(words)
# Compute entropy of the guess
wordle.compute_entropy_guess(word_played)  # make a guess
wordle.entropy

['abbey', 'abled', 'adept', 'after', 'agent', 'ahead', 'aider', 'alert', 'alien', 'alley', 'alter', 'amber', 'amend', 'angel', 'anger', 'annex', 'apnea', 'arena', 'ashen', 'askew', 'asset', 'avert', 'bagel', 'baker', 'baler', 'beach', 'beady', 'beard', 'beast', 'beaut', 'beech', 'beefy', 'befit', 'began', 'begat', 'beget', 'begin', 'begun', 'being', 'belch', 'belly', 'below', 'bench', 'beret', 'berry', 'berth', 'beset', 'betel', 'bevel', 'bezel', 'bicep', 'bleak', 'bleat', 'bleed', 'bleep', 'blend', 'bless', 'bluer', 'boney', 'bowel', 'boxer', 'bread', 'break', 'breed', 'brief', 'bused', 'buyer', 'cadet', 'cagey', 'camel', 'cameo', 'caper', 'cater', 'cedar', 'cello', 'cheap', 'cheat', 'check', 'cheek', 'cheer', 'chess', 'chest', 'chief', 'cider', 'clean', 'clear', 'cleat', 'cleft', 'clerk', 'clued', 'comet', 'corer', 'coven', 'cover', 'covet', 'covey', 'cower', 'creak', 'cream', 'credo', 'creed', 'creek', 'creep', 'crept', 'cress', 'crest', 'cried', 'crier', 'cruel', 'cyber', 'dealt', 

3.9324543321266496

In [ ]:
word_played = "crene"
green_letters = ""
yellow_letters = "ee"
bad_letters = ""

# Get index of each letter in the word
yellow_letter_positions = [
    index for index, letter in enumerate(word_played) if letter in yellow_letters
]
yellow_letter_positions

[2, 4]

In [2]:
import math
from collections import defaultdict, Counter

# words = load_words(FILE_PATH)
words = ["about", "blink", "crane", "drove", "merge", "rayon"]
secret = "rayon"
word_played = "crane"

# hash map of the feedback pattern
feedback_patern = defaultdict(list)
for word in words:
    for letter in word_played:
        if letter not in word:
            feedback_patern[word].append("gray")
        elif letter in word and word_played.index(letter) != word.index(letter):
            feedback_patern[word].append("yellow")
        elif letter in word_played and word_played.index(letter) == word.index(letter):
            feedback_patern[word].append("green")

# count the number of each feedback pattern
list_count = Counter(tuple(lst) for lst in feedback_patern.values())
# calculate the probabilities of each feedback pattern
probabilities = {}
for lst, count in list_count.items():
    probabilities[lst] = count / len(words)


def compute_entropy(probabilities):
    entropy = 0
    for prob in probabilities.values():
        entropy += -prob * math.log2(prob)
    return entropy


entropy = compute_entropy(probabilities)
entropy

2.584962500721156